In [ ]:
import requests
import hashlib
from datetime import datetime
import time
import random
import os
import csv
from bs4 import BeautifulSoup
import re 
import pandas as pd

# Parsing functions

In [ ]:
def parse_listing_html(text):
    #define the html text
    page_html = BeautifulSoup(text, 'html.parser')

    listing_title = page_html.find('span',attrs={'id':'titletextonly'}).text

    listing_date = page_html.find('time', class_='date timeago')['datetime']
    listing_price = page_html.find('span', class_='price')

    listing_bedbath= page_html.find('span', class_='shared-line-bubble').text

# Extract sqft info from bubbles class and use regex to find the digits (exclude the last number 2 from ft2)
    bubbles_sqft = page_html.findAll('span', class_='shared-line-bubble')
    listing_sqft = int("".join(re.findall("\d", str(bubbles_sqft[1]))[:-1]))

    listing_address = page_html.find('div', class_='mapaddress').text

    listing_info = page_html.findAll('p', class_='attrgroup')[1].text

    listing_body = page_html.find('section',attrs={'id':'postingbody'}).text
    
    listing_id = page_html.findAll('p', class_='postinginfo')[1].text

    return {
        'listing_title': listing_title,
        'listing_date':listing_date,
        'listing_price': int(listing_price.text.strip('$,').replace(",","")),
        'listing_bedbath':listing_bedbath,
        'listing_sqft':listing_sqft,
        'listing_address':listing_address,
        'listing_info':listing_info,
        'listing_body':listing_body,
        'listing_id':listing_id
    }


list_of_dicts = []
path = "/Users/pandabear/springboard/CapstoneTwoProject/data/raw"
dir_list = os.listdir(path)
# print(dir_list)
for folder in dir_list:
    if len(folder) == 19:
        path_html = path + '/' + folder
        file_list = os.listdir(path_html)
        #print(file_list)
        
        for file in file_list:
            if len(file) > 15:
                with open(path_html + '/' + file, 'r', encoding='windows-1252') as f:
                    text = f.read()
                    list_of_dicts.append(parse_listing_html(text))

                listing_df = pd.DataFrame(list_of_dicts)


In [ ]:
print(listing_df.head())

In [ ]:
#neighborhoods in SF
SF_neighborhoods = ['alamo square / nopa', 'bayview', 'bernal heights', 'castro / upper market', 'cole valley / ashbury hts', 'downtown / civic / van ness'
 'excelsior / outer mission', 'financial district', 'glen park', 'haight ashbury', 'hayes valley', 'ingleside / SFSU / CCSF', 'inner richmond'
 'inner sunset / UCSF', 'laurel hts / presidio', 'lower haight', 'lower nob hill', 'lower pac hts', 'marina / cow hollow', 'mission district'
 'nob hill', 'noe valley', 'north beach / telegraph hill', 'pacific heights', 'portola district', 'potrero hill', 'richmond / seacliff', 'russian hill', 'SOMA / south beach', 'sunset / parkside', 'tenderloin'
 'treasure island', 'twin peaks / diamond hts', 'USF / panhandle', 'visitacion valley', 'west portal / forest hill', 'western addition']